Create database and table

In [1]:
import sqlite3

conn = sqlite3.connect('example.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS users
                  (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')
conn.commit()
conn.close()

Insert data

In [2]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

cursor.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Alice", 25))
conn.commit()
conn.close()

Query data

In [3]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM users")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Name: {row[1]}, Age: {row[2]}")

conn.close()

ID: 1, Name: Alice, Age: 25


Parameterized queries and transactions

In [4]:
def add_users(users):
    conn = None
    try:
        conn = sqlite3.connect('example.db')
        cursor = conn.cursor()
        
        conn.execute("BEGIN TRANSACTION")
        cursor.executemany("INSERT INTO users (name, age) VALUES (?, ?)", users)
        conn.commit()
        print(f"Added {len(users)} users successfully")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

users = [("Bob", 30), ("Charlie", 35), ("Diana", 28)]
add_users(users)

Added 3 users successfully


Database class with context manager

In [5]:
class Database:
    def __init__(self, db_name):
        self.db_name = db_name
        
    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        return self.conn.cursor()
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.conn.commit()
        else:
            self.conn.rollback()
        self.conn.close()

# Usage
with Database('example.db') as cursor:
    cursor.execute("SELECT * FROM users")
    print(cursor.fetchall())

[(1, 'Alice', 25), (2, 'Bob', 30), (3, 'Charlie', 35), (4, 'Diana', 28)]
